In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

conn = sqlite3.connect('example.db')


def rsq(query: str) -> pd.DataFrame:
    return pd.read_sql_query(query, conn)


In [4]:
# "DateText"
# "2019-04-01 18:08:19.290
# "2019-04-07 06:14:30
# "2017-03-01
# "April 4, 2016
# "2017-06-14 09:30:00.0068848
# "19970614
# "2004-09-14T21:03:06Z"

In [5]:
df = pd.read_csv('./ImportedTimeTable.csv')
# df = pd.DataFrame({
#     "DateText": [
#         "2019-04-01 18:08:19.290",
#         "2019-04-07 06:14:30",
#         "2017-03-01",
#         "April 4, 2016",
#         "2017-06-14 09:30:00.0068848",
#         "19970614",
#         "2004-09-14T21:03:06Z"
#     ]
# })

df.to_sql('ImportedTime', conn, if_exists='replace', index=False)

7

In [ ]:
DECLARE @SomeTime DATETIME2(7) = SYSUTCDATETIME();

-- Retrieve  year, month, and day 
SELECT YEAR(@SomeTime) AS Year, MONTH(@SomeTime) AS Month, DAY(@SomeTime) AS Day;

In [3]:
rsq("""DECLARE
	@BerlinWallFalls DATETIME2(7) = '1989-11-09 23:49:36.2294852';

-- Fill in each date part
SELECT
	DATEPART(Year, @BerlinWallFalls) AS Year,
	DATEPART(DayOfYear, @BerlinWallFalls) AS DayOfYear,
	DATEPART(Day, @BerlinWallFalls) AS Day,
	DATEPART(Week, @BerlinWallFalls) AS Week,
	DATEPART(Second, @BerlinWallFalls) AS Second,
	DATEPART(Nanosecond, @BerlinWallFalls) AS Nanosecond;""")

DatabaseError: Execution failed on sql 'DECLARE
	@BerlinWallFalls DATETIME2(7) = '1989-11-09 23:49:36.2294852';

-- Fill in each date part
SELECT
	DATEPART(Year, @BerlinWallFalls) AS Year,
	DATEPART(DayOfYear, @BerlinWallFalls) AS DayOfYear,
	DATEPART(Day, @BerlinWallFalls) AS Day,
	DATEPART(Week, @BerlinWallFalls) AS Week,
	DATEPART(Second, @BerlinWallFalls) AS Second,
	DATEPART(Nanosecond, @BerlinWallFalls) AS Nanosecond;': near "DECLARE": syntax error

In [41]:
rsq("""
    SELECT
        strftime('%Y', '1989-11-09 23:49:36.2294852') AS Year,
        strftime('%j', '1989-11-09 23:49:36.2294852') AS DayOfYear,
        strftime('%d', '1989-11-09 23:49:36.2294852') AS Day,
        strftime('%W', '1989-11-09 23:49:36.2294852') AS Week,
        strftime('%S', '1989-11-09 23:49:36.2294852') AS Second,
        strftime('%f', '1989-11-09 23:49:36.2294852') AS Nanosecond;
        """)


,Year,DayOfYear,Day,Week,Second,Nanosecond
0,1989,313,09,45,36,36.229


In [ ]:
DECLARE
	@LeapDay DATETIME2(7) = '2012-02-29 18:00:00';
-- Fill in the date parts and intervals as needed 
SELECT DATEADD(DAY, -1, @LeapDay) AS PriorDay, 
DATEADD(DAY, 1, @LeapDay) AS NextDay, 
-- For leap years, we need to move 4 years, not just 1 
DATEADD(YEAR, -4, @LeapDay) AS PriorLeapYear, DATEADD(YEAR, 4, @LeapDay) AS NextLeapYear, DATEADD(YEAR, -1, @LeapDay) AS PriorYear;

In [44]:
rsq("""SELECT DATE('2012-02-28 18:00:00') AS PriorDay,
DATE('2012-03-01 18:00:00') AS NextDay,
DATE('2008-02-29 18:00:00') AS PriorLeapYear,
DATE('2016-02-29 18:00:00') AS NextLeapYear,
DATE('2012-02-28 18:00:00') AS PriorYear;""")

,PriorDay,NextDay,PriorLeapYear,NextLeapYear,PriorYear
0,2012-02-28,2012-03-01,2008-02-29,2016-02-29,2012-02-28


In [ ]:
DECLARE
	@PostLeapDay DATETIME2(7) = '2012-03-01 18:00:00';

-- Fill in the date parts and intervals as needed
SELECT
	DATEADD(DAY, -1, @PostLeapDay) AS PriorDay,
	DATEADD(DAY, 1, @PostLeapDay) AS NextDay,
	DATEADD(YEAR, -4, @PostLeapDay) AS PriorLeapYear,
	DATEADD(YEAR, 4, @PostLeapDay) AS NextLeapYear,
	DATEADD(YEAR, -1, @PostLeapDay) AS PriorYear,
    -- Move 4 years forward and one day back
DATEADD(DAY, -1, DATEADD(YEAR, 4, @PostLeapDay)) AS NextLeapDay, DATEADD(DAY, -2, @PostLeapDay) AS TwoDaysAgo;

In [43]:
rsq("""
-- Fill in the date parts and intervals as needed
SELECT
	DATE('2012-03-01 18:00:00', '-1 day') AS PriorDay,
	DATE('2012-03-01 18:00:00', '+1 day') AS NextDay,
	DATE('2012-03-01 18:00:00', '-4 year') AS PriorLeapYear,
	DATE('2012-03-01 18:00:00', '+4 year') AS NextLeapYear,
	DATE('2012-03-01 18:00:00', '-1 year') AS PriorYear,
    -- Move 4 years forward and one day back
	DATE(DATE('2012-03-01 18:00:00', '+4 year'), '-1 day') AS NextLeapDay, 
	DATE('2012-03-01 18:00:00', '-2 day') AS TwoDaysAgo; 
 """)

,PriorDay,NextDay,PriorLeapYear,NextLeapYear,PriorYear,NextLeapDay,TwoDaysAgo
0,2012-02-29,2012-03-02,2008-03-01,2016-03-01,2011-03-01,2016-02-29,2012-02-28


In [ ]:
DECLARE
	@PostLeapDay DATETIME2(7) = '2012-03-01 18:00:00',
    @TwoDaysAgo DATETIME2(7);

SELECT
	@TwoDaysAgo = DATEADD(DAY, -2, @PostLeapDay);

SELECT
	@TwoDaysAgo AS TwoDaysAgo,
	@PostLeapDay AS SomeTime,
    -- Fill in the appropriate function and date types
	DATEDIFF(DAY, @TwoDaysAgo, @PostLeapDay) AS DaysDifference, DATEDIFF(HOUR, @TwoDaysAgo, @PostLeapDay) AS HoursDifference, DATEDIFF(MINUTE, @TwoDaysAgo, @PostLeapDay) AS MinutesDifference;

In [5]:
rsq("""SELECT CAST('2016-11-03 00:30:29.245' AS NVARCHAR(30)) AS DefaultForm, 
    CAST('2016-11-03 00:30:29.245' AS NVARCHAR(30)) AS UK_dmy, 
    CAST('2016-11-03 00:30:29.245' AS NVARCHAR(30)) AS US_mdy, 
    CAST('2016-11-03 00:30:29.245' AS NVARCHAR(30)) AS UK_dmyyyy, 
    CAST('2016-11-03 00:30:29.245' AS NVARCHAR(30)) AS US_mdyyyy;""")


,DefaultForm,UK_dmy,US_mdy,UK_dmyyyy,US_mdyyyy
0,2016-11-03 00:30:29.245,2016-11-03 00:30:29.245,2016-11-03 00:30:29.245,2016-11-03 00:30:29.245,2016-11-03 00:30:29.245


In [8]:
rsq("""
SELECT
	-- Fill in the function call and format parameter
	strftime('%m/%d/%Y', '2008-12-03 19:45:00.033') AS US_d,
	strftime('%d.%m.%Y', '2008-12-03 19:45:00.033') AS DE_d,
	-- Fill in the locale for Japan
	strftime('%Y/%m/%d', '2008-12-03 19:45:00.033') AS JP_d,
	strftime('%Y-%m-%d', '2008-12-03 19:45:00.033') AS CN_d;
""")


,US_d,DE_d,JP_d,CN_d
0,12/03/2008,03.12.2008,2008/12/03,2008-12-03


In [ ]:
DECLARE
	@Python3ReleaseDate DATETIME2(3) = '2008-12-03 19:45:00.033';
    
SELECT -- 20081203 
FORMAT(@Python3ReleaseDate, 'yyyyMMdd') AS F1, -- 2008-12-03 
FORMAT(@Python3ReleaseDate, 'yyyy-MM-dd') AS F2, -- Dec 03+2008 (the + is just a "+" character) 
FORMAT(@Python3ReleaseDate, 'MMM dd+yyyy') AS F3, -- 12 08 03 (month, two-digit year, day) 
FORMAT(@Python3ReleaseDate, 'MM yy dd') AS F4, -- 03 Dec 07:45 2008.00 -- (day, hour, minute, year, ".", second) 
FORMAT(@Python3ReleaseDate, 'dd hh:mm yyyy.ss') AS F5;

In [13]:
rsq("""SELECT 
    strftime('%Y%m%d', '2008-12-03 19:45:00.033') AS F1, 
    strftime('%Y-%m-%d', '2008-12-03 19:45:00.033') AS F2, 
    strftime('%b %d+%Y', '2008-12-03 19:45:00.033') AS F3, 
    strftime('%m %y %d', '2008-12-03 19:45:00.033') AS F4, 
    strftime('%d %H:%M %Y.%f', '2008-12-03 19
    :45:00.033') AS F5;""")


,F1,F2,F3,F4,F5
0,20081203,2008-12-03,None,None,03 19:45 2008.00.033


In [ ]:
rsq("""-- Find Tuesdays in December for calendar years 2008-2010 
SELECT c.Date FROM dbo.Calendar c WHERE c.MonthName = 'December' AND c.DayName = 'Tuesday' AND c.CalendarYear BETWEEN 2008 AND 2010 ORDER BY c.Date;""")

In [27]:
rsq("""-- Find Tuesdays in December for calendar years 2008-2010 
SELECT c.Date FROM Calendar c WHERE c.MonthName = 'December' AND c.DayName = 'Tuesday' AND c.CalendarYear BETWEEN 2008 AND 2010 ORDER BY c.Date;""")

,Date
0,2008-12-02
1,2008-12-09
2,2008-12-16
3,2008-12-23
4,2008-12-30
5,2009-12-01
6,2009-12-08
7,2009-12-15
8,2009-12-22
9,2009-12-29


In [26]:
rsq("""-- Find fiscal week 29 of fiscal year 2019
 SELECT c.Date FROM Calendar c WHERE -- Instead of month, use the fiscal week 
 c.FiscalWeekOfYear = 29 -- Instead of calendar year, use fiscal year 
 AND c.FiscalYear = 2019 ORDER BY c.Date ASC;""")

,Date
0,2020-01-12
1,2020-01-13
2,2020-01-14
3,2020-01-15
4,2020-01-16
5,2020-01-17
6,2020-01-18


In [ ]:
# SELECT
# 	ir.IncidentDate,
# 	c.FiscalDayOfYear,
# 	c.FiscalWeekOfYear
# FROM dbo.IncidentRollup ir INNER JOIN dbo.Calendar c ON ir.IncidentDate = c.Date WHERE -- Incident type 3 
# ir.IncidentTypeID = 3 -- Fiscal year 2019 
# AND c.FiscalYear = 2019 -- Fiscal quarter 3 
# AND c.FiscalQuarter = 3;

In [31]:
rsq("""SELECT
	ir.IncidentDate,
	c.FiscalDayOfYear,
	c.FiscalWeekOfYear
FROM IncidentRollup ir INNER JOIN Calendar c ON ir.IncidentDate = c.Date WHERE -- Incident type 3 
ir.IncidentTypeID = 3 -- Fiscal year 2019 
AND c.FiscalYear = 2019 -- Fiscal quarter 3 
AND c.FiscalQuarter = 3;""")

,IncidentDate,FiscalDayOfYear,FiscalWeekOfYear
0,2020-01-02,186,27
1,2020-01-05,189,28
2,2020-01-06,190,28
3,2020-01-08,192,28
4,2020-01-09,193,28
5,2020-01-10,194,28
6,2020-01-28,212,31
7,2020-01-31,215,31
8,2020-02-02,217,32
9,2020-02-03,218,32


In [ ]:
#  SELECT
# 	ir.IncidentDate,
# 	c.FiscalDayOfYear,
# 	c.FiscalWeekOfYear
# FROM dbo.IncidentRollup ir
# 	INNER JOIN dbo.Calendar c
# 		ON ir.IncidentDate = c.Date
# WHERE
#     -- Incident type 4
# 	ir.IncidentTypeID = 4
#     -- Fiscal year 2019
# 	AND c.FiscalYear = 2019
#     -- Beyond fiscal week of year 30
# 	AND c.FiscalWeekOfYear > 30
#     -- Only return weekends
# 	AND c.IsWeekend = 1;

In [35]:
rsq("""SELECT
    ir.IncidentDate,
    c.FiscalDayOfYear,
    c.FiscalWeekOfYear
FROM IncidentRollup ir
    INNER JOIN Calendar c
        ON ir.IncidentDate = c.Date
WHERE ir.IncidentTypeID = 4 -- Incident type 4 
    AND c.FiscalYear = 2019 -- Fiscal year 2019 
    AND c.FiscalWeekOfYear > 30 -- Beyond fiscal week of year 30 
    AND c.IsWeekend = 1; -- Only return weekends
""")


,IncidentDate,FiscalDayOfYear,FiscalWeekOfYear
0,2020-02-16,231,34
1,2020-02-23,238,35
2,2020-03-21,265,38
3,2020-03-29,273,40
4,2020-04-05,280,41
5,2020-05-23,328,47
6,2020-06-21,357,52
7,2020-06-28,364,53
